In [1]:
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain, RetrievalQA, ConversationChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate, load_prompt
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.llms import AzureOpenAI
from PyPDF2 import PdfReader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Replicate
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
import replicate
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
#dotenv.load_dotenv()

#prompt manual
templatemaster = """Você é um assistente prestativo, respeitoso e honesto. Sempre responda da maneira mais prestativa possível, estando seguro.
Suas respostas não devem incluir conteúdo prejudicial, antiético, racista, sexista, tóxico, perigoso ou ilegal.
Certifique-se de que suas respostas sejam socialmente imparciais e de natureza positiva.
Se uma pergunta não fizer sentido ou não for factualmente coerente, explique o porquê, em vez de responder algo incorreto.
Se você não sabe a resposta a uma pergunta, não compartilhe informações falsas.
salvarar o chat history (delimited by <hs></hs>)

Question: {input}

------
<hs>
{history}
</hs>
------
Answer: """
prompt = PromptTemplate(
    template=templatemaster,
    input_variables=['input','history']
)


template = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt1 = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [3]:
#LMM  configurado, juntamente com .env, Favor não mudar nada.
llm = AzureChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_DEPLOYMENT_VERSION"),
    temperature=0.7,
    max_retries=1,
    max_tokens=50)

# um modelo de Chain usando LLMChain, tem outro que podemos usar...  Tem que ser o de busca.
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False
)

# embedding
embedding = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment=os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_type="azure",
    chunk_size=1
)


In [4]:
conversation = ConversationChain(llm=llm, verbose=True, memory=ConversationBufferMemory(), prompt=prompt)

In [5]:
conversation('Se eu for add?')



> Entering new ConversationChain chain...
Prompt after formatting:
Você é um assistente prestativo, respeitoso e honesto. Sempre responda da maneira mais prestativa possível, estando seguro.
Suas respostas não devem incluir conteúdo prejudicial, antiético, racista, sexista, tóxico, perigoso ou ilegal.
Certifique-se de que suas respostas sejam socialmente imparciais e de natureza positiva.
Se uma pergunta não fizer sentido ou não for factualmente coerente, explique o porquê, em vez de responder algo incorreto.
Se você não sabe a resposta a uma pergunta, não compartilhe informações falsas.
salvarar o chat history (delimited by <hs></hs>)

Question: Se eu for add?

------
<hs>

</hs>
------
Answer: 

> Finished chain.


{'input': 'Se eu for add?',
 'history': '',
 'response': 'Desculpe, eu não entendi sua pergunta. Poderia reformulá-la, por favor?'}

In [16]:
def count_token(result):
    with get_openai_callback() as cb:
        #result = llm_chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
        #print(result)
    return result

def get_pdf_text(pdf_file):
    text = ""
    pdf_reader = PdfReader(pdf_file)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


def PDF_READER(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunk = text_splitter.split_documents(text) or text_splitter.split_text(text)
    return chunk


def query(q):
    print("Query: ", q)
    print("Answer: ", qa.run(q))




In [17]:
loader = PyPDFLoader("Tormenta20.pdf")
docs = loader.load()

ValueError: File path Tormenta20.pdf is not a valid file or url

In [ ]:
persist_directory = "E:\GPT-LLM-ESTUDOS\Desafiogpt\Code\Docsnotion\docs"

In [ ]:
chunk = PDF_READER(docs)
#vectordb  = Chroma.from_documents(chunk, embedding)
vectordb = Chroma.from_documents(chunk, embedding)
#memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history", human_prefix='friend')
#qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever(), memory=memory)
#conversation = ConversationChain(llm=llm, verbose=True, memory=ConversationBufferMemory())

In [ ]:
query("Uma aventura pode ser especial?")

Query:  Uma aventura pode ser especial?


NameError: name 'qa' is not defined

In [ ]:

#vectordb = vectordb.similarity_search(query)
#vectores = Chroma.from_documents(texts, embeddings)
#vectores = FAISS.from_documents(texts, embeddings)


In [ ]:


llm2 = Replicate(
        streaming = True,
        model = "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", 
        callbacks=[StreamingStdOutCallbackHandler()],
        input = {"temperature": 0.1, "max_length" :50,"top_p":1, "min_new_tokens": -1})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=llm2, prompt=prompt, verbose=True)



Init param `input` is deprecated, please use `model_kwargs` instead.


In [ ]:
chain("oi")



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.

Question: oi

Answer: 


KeyboardInterrupt: 

In [ ]:
import streamlit as st
import logging
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from streamlit_chat import message
import os
from dotenv import load_dotenv

load_dotenv()
# Prompt manual
template = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# Configuração do modelo de Chat (AzureChatOpenAI)
llm = AzureChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_DEPLOYMENT_VERSION"),
    temperature=0,
    max_retries=1,
    max_tokens=100
)

# Configuração do embedding
embedding = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment=os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_type="azure",
    chunk_size=1
)

# Carregar documentos PDF
raw_documento_OC = PyPDFLoader(".\docs\Eccosys -.pdf").load()
raw_documento_DC = PyPDFLoader(".\docs\Eccosys -.pdf").load()
raw_documento_LEVE = PyPDFLoader(".\docs\Eccosys -.pdf").lazy_load()

# Função para dividir o texto do PDF em chunks
def text_splitter_Pdf(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunk = text_splitter.split_documents(text)
    return chunk

# Dividir documentos em chunks
chunk_OC = text_splitter_Pdf(raw_documento_OC)
chunk_DC = text_splitter_Pdf(raw_documento_DC)
chunk_LEVE = text_splitter_Pdf(raw_documento_LEVE)

persist_directory = "./vectorstore"


def generate_response(user_query, selected_pdfs):
    context = []  # Inicialize a lista de contexto
    for pdf, selected in zip([chunk_DC, chunk_LEVE, chunk_OC], selected_pdfs):
        if selected:
            context.extend(pdf)
    if not context:
        st.error("Nenhum PDF selecionado. Escolha pelo menos um.")
        return
    
    memory = ConversationBufferMemory(memory_key='messages', return_messages=False)
    vectordb = Chroma.from_documents(chunk_DC, embedding, persist_directory=persist_directory)
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=vectordb.as_retriever(),
        verbose=True, chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"),
    }
)


    # Substitua 'qa' pela instância real do seu RetrievalQA
    result = qa(user_query)
    return result.get("result")

# Função principal
def main():
    # Defina as cores da página usando a paleta de cores do Streamlit
    st.set_page_config(
        page_title="LangChain + Streamlit Chatbot",
        page_icon="✨",
        layout="wide",
        initial_sidebar_state="expanded",
    )

    # Título da página
    st.title("LangChain + Streamlit Chatbot")

    # Checkboxes para selecionar PDFs
    selected_pdfs = st.checkbox("Usar DC", value=True), st.checkbox("Usar LEVE"), st.checkbox("Usar OC")


    # Título adicional
    st.title("Chat with your CSV Data")

# https://python.langchain.com/docs/integrations/memory/streamlit_chat_message_history
    
    
    # Verifique se existe uma lista de mensagens na sessão do Streamlit
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Exiba as mensagens anteriores
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])

    # Entrada do usuário
    if prompt := st.chat_input("IA DE SUP"):
        st.session_state.messages.append({"role": "user", "content": prompt})

        # Exiba a mensagem do usuário
        with st.chat_message("user"):
            st.write(prompt)

        # Exiba a resposta do assistente com um spinner durante o processamento
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            with st.spinner(text="Thinking..."):
                assistant_response = generate_response(prompt, selected_pdfs)
                message_placeholder.write(assistant_response)

        # Adicione a resposta do assistente à lista de mensagens
        st.session_state.messages.append({"role": "assistant", "content": assistant_response})

        st.write(st.session_state.messages)

# Execute a função principal
if __name__ == "__main__":
    main()